**Author:** Lisa Wallner  
**Description:** In this notebook a file with the relevant metadata of multiple GitHub repositories will be created. Keep in mind, the files are already created! If you want to test the notebook create new filenames, for example *../data/df_repos_metadata_star_up_to_max_TEST.json*  
**Depencencies:**  
+ data/raw_data_zip/raw_data_no_range.zip  
+ data/raw_data_zip/raw_data_0_22196.zip  

*Hint: If lines are created with support of a Large Language Model or the code is taken from another source, you find following hint at the end of the line: (generated with Microsoft Copilot) or (source: link_to_source)*

In [22]:
import pandas as pd
import json
from pathlib import Path
from langdetect import detect
import zipfile
import os

In [23]:
def load_json(path):
    with open(path, 'r') as file:
        loaded_data = json.load(file)
    
    return loaded_data

In [24]:
def rezip_files(path, range_type):
    
    original_zip = path # path to the original ZIP file
    if range_type == 'range':
        extracted_dir = '../data/raw_data/range' # directory where the extract contents are saved
    else:
        extracted_dir = '../data/raw_data/no_range'

    with zipfile.ZipFile(original_zip, 'r') as zip_ref: 
        zip_ref.extractall(extracted_dir) # extract all files

In [25]:
# columns which are requiered for preprocessing and further steps
columns = [
    'id', 
    'name', 
    'full_name', 
    'html_url', 
    'description', 
    'url', 
    'labels_url', 
    'created_at', 
    'updated_at', 
    'pushed_at', 
    'size', 
    'stargazers_count', 
    'watchers_count', 
    'language', 
    'has_issues', 
    'has_projects', 
    'has_downloads', 
    'has_wiki', 
    'has_pages', 
    'has_discussions', 
    'forks_count', 
    'open_issues_count', 
    'license', 
    'allow_forking', 
    'topics', 
    'visibility', 
    'forks', 
    'open_issues', 
    'watchers', 
    'default_branch', 
    'score'
]

### Create two paths to save .json

In [ ]:
# file with repos up to 21947 stars
file_no_range = '../data/df_repos_metadata_star_up_to_max.json'
# file with repos 0 to 21947 stars
file_range = '../data/df_repos_metadata_star_0_to_21947.json'

### Get column names for metadata file

In [27]:
help_columns = load_json(path='../data/helper/help_columns.json')
keys = list(help_columns[0].keys()) # get keys of loaded_data as list

In [28]:
len(keys)

82

### Get paths of raw data

In [29]:
# loaded zip files of metadata in raw_data
path_no_range = '../data/raw_data_zip/raw_data_no_range.zip'
path_range = '../data/raw_data_zip/raw_data_range_0_22196.zip'

# was executed before
# rezip_files(path=path_no_range, range_type='no_range')
# rezip_files(path=path_range, range_type='range')

In [30]:
path_range = Path('../data/raw_data/range') 
all_files_range = [file.name for file in path_range.iterdir() if file.is_file()]

path_no_range = Path('../data/raw_data/no_range') 
all_files_no_range = [file.name for file in path_no_range.iterdir() if file.is_file()]

### Open all jsons and load repo metadata into dataframe

In [31]:
# create empty df with keys of loaded_data as columns
df_raw_range = pd.DataFrame(columns=keys)
df_raw_no_range = pd.DataFrame(columns=keys)

In [32]:
for file in all_files_range:
    data = load_json(path=f'../data/raw_data/range/{file}')
    #  iterate through subdictionary in data and concatenate the content of the subdictionary to df_repos
    for repo in data:
        # create tmp df_repo for each repo
        df_tmp = pd.DataFrame(data=[repo], columns=keys)
        # concatenate df_repos with df_repo
        df_raw_range = pd.concat([df_raw_range, df_tmp], ignore_index=True)

/tmp/ipykernel_15575/967994083.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_raw_range = pd.concat([df_raw_range, df_tmp], ignore_index=True)


In [33]:
for file in all_files_no_range:
    data = load_json(path=f'../data/raw_data/no_range/{file}')
    data = data['items']
    #  iterate through subdictionary in data and concatenate the content of the subdictionary to df_repos
    for repo in data:
        # create tmp df_repo for each repo
        df_tmp = pd.DataFrame(data=[repo], columns=keys)
        # concatenate df_repos with df_repo
        df_raw_no_range = pd.concat([df_raw_no_range, df_tmp], ignore_index=True)

/tmp/ipykernel_15575/2625439888.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_raw_no_range = pd.concat([df_raw_no_range, df_tmp], ignore_index=True)


### Clean remove unnecessary columns from dataframes

In [34]:
for k in keys:
    if k in columns:
        continue
    else:
        df_raw_range = df_raw_range.drop([k], axis=1)

In [35]:
df_raw_range.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591 entries, 0 to 590
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 591 non-null    object 
 1   name               591 non-null    object 
 2   full_name          591 non-null    object 
 3   html_url           591 non-null    object 
 4   description        579 non-null    object 
 5   url                591 non-null    object 
 6   labels_url         591 non-null    object 
 7   created_at         591 non-null    object 
 8   updated_at         591 non-null    object 
 9   pushed_at          591 non-null    object 
 10  size               591 non-null    object 
 11  stargazers_count   591 non-null    object 
 12  watchers_count     591 non-null    object 
 13  language           591 non-null    object 
 14  has_issues         591 non-null    object 
 15  has_projects       591 non-null    object 
 16  has_downloads      591 non

In [36]:
for k in keys:
    if k in columns:
        continue
    else:
        df_raw_no_range = df_raw_no_range.drop([k], axis=1)

In [37]:
df_raw_no_range['stargazers_count'].describe()

count     1050
unique     991
top       7267
freq         3
Name: stargazers_count, dtype: int64

### Save each dataframe in single JSON file

In [ ]:
tmp_json = df_raw_range.to_json(orient='records', lines=False, force_ascii=False)

# '../data/df_repos_metadata_star_0_to_22196.json'
with open(file_range, 'w') as file:
   file.write(tmp_json)

In [ ]:
tmp_json = df_raw_no_range.to_json(orient='records', lines=False, force_ascii=False)

# '../data/df_repos_metadata_star_up_to_max.json'
with open(file_no_range, 'w') as file:
   file.write(tmp_json)

### Load each JSON and prepare for concatinatig

In [ ]:
with open('../data/df_repos_metadata_star_up_to_max.json', 'r') as file: # JSON file is loaded and it contents saved within a variable
    loaded_data = json.load(file)

df1 = pd.DataFrame(loaded_data)

In [41]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 1050 non-null   int64  
 1   name               1050 non-null   object 
 2   full_name          1050 non-null   object 
 3   html_url           1050 non-null   object 
 4   description        1038 non-null   object 
 5   url                1050 non-null   object 
 6   labels_url         1050 non-null   object 
 7   created_at         1050 non-null   object 
 8   updated_at         1050 non-null   object 
 9   pushed_at          1050 non-null   object 
 10  size               1050 non-null   int64  
 11  stargazers_count   1050 non-null   int64  
 12  watchers_count     1050 non-null   int64  
 13  language           1050 non-null   object 
 14  has_issues         1050 non-null   bool   
 15  has_projects       1050 non-null   bool   
 16  has_downloads      1050 

In [42]:
df1.describe()

,id,size,stargazers_count,watchers_count,forks_count,open_issues_count,forks,open_issues,watchers,score
count,1.050000e+03,1.050000e+03,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000,1050.0
mean,2.842129e+08,1.316390e+05,21701.457143,21701.457143,3555.066667,375.716190,3555.066667,375.716190,21701.457143,1.0
std,2.800213e+08,4.986652e+05,29908.569921,29908.569921,5971.414570,1003.159613,5971.414570,1003.159613,29908.569921,0.0
min,2.655400e+04,7.000000e+00,7193.000000,7193.000000,108.000000,0.000000,108.000000,0.000000,7193.000000,1.0
25%,4.880424e+07,5.378500e+03,8829.500000,8829.500000,989.000000,56.000000,989.000000,56.000000,8829.500000,1.0
50%,1.615837e+08,2.407950e+04,12306.500000,12306.500000,1814.000000,139.500000,1814.000000,139.500000,12306.500000,1.0
75%,5.702785e+08,8.391375e+04,21947.250000,21947.250000,3423.000000,331.000000,3423.000000,331.000000,21947.250000,1.0
max,9.548733e+08,1.087097e+07,335520.000000,335520.000000,49242.000000,16073.000000,49242.000000,16073.000000,335520.000000,1.0


In [ ]:
quantile_75 = df1['stargazers_count'].describe().loc['75%']
quantile_75

In [47]:
len(df1[df1['stargazers_count'] > quantile_75])

263

In [ ]:
df1 = df1[df1['stargazers_count'] > quantile_75]

In [49]:
len(df1)

263

In [ ]:
with open('../data/df_repos_metadata_star_0_to_22196.json', 'r') as file: # JSON file is loaded and it contents saved within a variable
    loaded_data = json.load(file)

df2 = pd.DataFrame(loaded_data)

In [44]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591 entries, 0 to 590
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 591 non-null    int64  
 1   name               591 non-null    object 
 2   full_name          591 non-null    object 
 3   html_url           591 non-null    object 
 4   description        579 non-null    object 
 5   url                591 non-null    object 
 6   labels_url         591 non-null    object 
 7   created_at         591 non-null    object 
 8   updated_at         591 non-null    object 
 9   pushed_at          591 non-null    object 
 10  size               591 non-null    int64  
 11  stargazers_count   591 non-null    int64  
 12  watchers_count     591 non-null    int64  
 13  language           591 non-null    object 
 14  has_issues         591 non-null    bool   
 15  has_projects       591 non-null    bool   
 16  has_downloads      591 non

In [45]:
df2.describe()

,id,size,stargazers_count,watchers_count,forks_count,open_issues_count,forks,open_issues,watchers,score
count,5.910000e+02,5.910000e+02,591.000000,591.000000,591.000000,591.000000,591.000000,591.000000,591.000000,591.0
mean,2.808887e+08,1.005392e+05,8885.094755,8885.094755,1319.659898,198.570220,1319.659898,198.570220,8885.094755,1.0
std,2.771278e+08,3.897973e+05,5747.753005,5747.753005,1223.648694,336.121202,1223.648694,336.121202,5747.753005,0.0
min,2.655400e+04,7.000000e+00,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.0
25%,4.512183e+07,2.661500e+03,4106.000000,4106.000000,446.500000,28.000000,446.500000,28.000000,4106.000000,1.0
50%,1.626909e+08,1.271800e+04,8233.000000,8233.000000,960.000000,87.000000,960.000000,87.000000,8233.000000,1.0
75%,5.619080e+08,5.053400e+04,13041.500000,13041.500000,1839.000000,220.000000,1839.000000,220.000000,13041.500000,1.0
max,9.548733e+08,6.384691e+06,22144.000000,22144.000000,7832.000000,2921.000000,7832.000000,2921.000000,22144.000000,1.0


### Combine the two dataframes into one for further analysis

In [50]:
df_repos = pd.DataFrame()
df_repos = pd.concat([df2, df1])

In [51]:
df_repos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 854 entries, 0 to 1019
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 854 non-null    int64  
 1   name               854 non-null    object 
 2   full_name          854 non-null    object 
 3   html_url           854 non-null    object 
 4   description        838 non-null    object 
 5   url                854 non-null    object 
 6   labels_url         854 non-null    object 
 7   created_at         854 non-null    object 
 8   updated_at         854 non-null    object 
 9   pushed_at          854 non-null    object 
 10  size               854 non-null    int64  
 11  stargazers_count   854 non-null    int64  
 12  watchers_count     854 non-null    int64  
 13  language           854 non-null    object 
 14  has_issues         854 non-null    bool   
 15  has_projects       854 non-null    bool   
 16  has_downloads      854 non-nul

### Remove duplicates

In [52]:
# the behavior of the api is sometimes confusing, there are some duplicates in the dataframe --> remove them
df_cleaned = df_repos.drop_duplicates(subset=['full_name'])

### Filter df_repos for spoken language 'English'

In [53]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return "error"

In [54]:
df_repos['language_spoken'] = df_repos['description'].apply(detect_language)

In [55]:
df_repos['language_spoken'].value_counts()

language_spoken
en       752
zh-cn     36
error     16
nl         7
ca         7
vi         6
da         6
fr         5
it         5
ko         4
no         3
af         2
ru         1
ro         1
et         1
es         1
cs         1
Name: count, dtype: int64

In [56]:
len(df_repos[df_repos['language_spoken'] == 'en'])

752

### Save final file for the repository metadata as JSON for further processing

In [57]:
tmp_json = df_repos.to_json(orient='records', lines=False, force_ascii=False)

In [ ]:
with open('../data/df_repos_metadata.json', 'w') as file:
   file.write(tmp_json)

### Load repository metadata (test)

In [ ]:
with open('../data/df_repos_metadata.json', 'r') as file:
    loaded_data = json.load(file)

In [60]:
df = pd.DataFrame(loaded_data)

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 854 entries, 0 to 853
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 854 non-null    int64  
 1   name               854 non-null    object 
 2   full_name          854 non-null    object 
 3   html_url           854 non-null    object 
 4   description        838 non-null    object 
 5   url                854 non-null    object 
 6   labels_url         854 non-null    object 
 7   created_at         854 non-null    object 
 8   updated_at         854 non-null    object 
 9   pushed_at          854 non-null    object 
 10  size               854 non-null    int64  
 11  stargazers_count   854 non-null    int64  
 12  watchers_count     854 non-null    int64  
 13  language           854 non-null    object 
 14  has_issues         854 non-null    bool   
 15  has_projects       854 non-null    bool   
 16  has_downloads      854 non